In [1]:
!pwd

/notebook/ColBERT


In [16]:
import sys
sys.path.append("/notebook/ColBERT")

import os
import random
import time
import torch
import torch.nn as nn
import numpy as np

from transformers import AdamW
from colbert.utils.runs import Run
from colbert.utils.amp import MixedPrecisionManager

from colbert.training.lazy_batcher import LazyBatcher
from colbert.training.eager_batcher import EagerBatcher
from colbert.parameters import DEVICE

from colbert.modeling.colbert import ColBERT
from colbert.utils.utils import print_message
from colbert.training.utils import print_progress, manage_checkpoints

from colbert.utils.tensor_net import TTLayer

query_maxlen = 512
query_maxlen = 512
doc_maxlen = 512
dim = 128
similarity = 'cosine'

colbert = ColBERT.from_pretrained('bert-base-uncased', query_maxlen=query_maxlen, doc_maxlen=doc_maxlen, dim=dim, similarity_metric=similarity, mask_punctuation=False)
print(sum(p.numel() for p in colbert.parameters()))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAI

109580544


In [2]:
TT_SHAPES = (32, 48, 48, 32)
TT_RANKS = [1, 380, 390, 380, 1] # comp rate 0.5
SVD_RANKS = 350


from tensor_net1 import TTLayer
for i in [0, 2, 4, 6, 8, 10]:
            # fc part
            fc_w = colbert.bert.encoder.layer[i].intermediate.dense
            fc_b = colbert.bert.encoder.layer[i].intermediate.dense.bias
            (out_, in_) = fc_w.weight.shape
            factorized_layer = TTLayer(fc_w, shapes = TT_SHAPES, in_dims = [32, 24], ranks = TT_RANKS)
            for elem in factorized_layer.cores:
                print (elem.shape)

            colbert.bert.encoder.layer[i].intermediate.dense = factorized_layer

            fc_w = colbert.bert.encoder.layer[i].output.dense
            factorized_layer = TTLayer(fc_w, shapes = TT_SHAPES, in_dims = [32, 48, 2], ranks = TT_RANKS)
            colbert.bert.encoder.layer[i].output.dense = factorized_layer

print(sum(p.numel() for p in colbert.parameters()))

layer.shape torch.Size([3072, 768])
torch.Size([1, 32, 32])
torch.Size([32, 48, 390])
torch.Size([390, 48, 32])
torch.Size([32, 32, 1])
layer.shape torch.Size([768, 3072])
layer.shape torch.Size([3072, 768])
torch.Size([1, 32, 32])
torch.Size([32, 48, 390])
torch.Size([390, 48, 32])
torch.Size([32, 32, 1])
layer.shape torch.Size([768, 3072])
layer.shape torch.Size([3072, 768])
torch.Size([1, 32, 32])
torch.Size([32, 48, 390])
torch.Size([390, 48, 32])
torch.Size([32, 32, 1])
layer.shape torch.Size([768, 3072])
layer.shape torch.Size([3072, 768])
torch.Size([1, 32, 32])
torch.Size([32, 48, 390])
torch.Size([390, 48, 32])
torch.Size([32, 32, 1])
layer.shape torch.Size([768, 3072])
layer.shape torch.Size([3072, 768])
torch.Size([1, 32, 32])
torch.Size([32, 48, 390])
torch.Size([390, 48, 32])
torch.Size([32, 32, 1])
layer.shape torch.Size([768, 3072])
layer.shape torch.Size([3072, 768])
torch.Size([1, 32, 32])
torch.Size([32, 48, 390])
torch.Size([390, 48, 32])
torch.Size([32, 32, 1])
laye

In [4]:
TT_SHAPES = (32, 48, 48, 32)
TT_RANKS = [1, 450, 450, 450, 1] # comp rate 0.5
SVD_RANKS = 350


from tensor_net1 import TTLayer
for i in [0, 2, 4, 6, 8, 10]:
            # fc part
            fc_w = colbert.bert.encoder.layer[i].intermediate.dense
            fc_b = colbert.bert.encoder.layer[i].intermediate.dense.bias
            (out_, in_) = fc_w.weight.shape
            factorized_layer = TTLayer(fc_w, shapes = TT_SHAPES, in_dims = [32, 24], ranks = TT_RANKS)
            for elem in factorized_layer.cores:
                print (elem.shape)

            colbert.bert.encoder.layer[i].intermediate.dense = factorized_layer

            fc_w = colbert.bert.encoder.layer[i].output.dense
            factorized_layer = TTLayer(fc_w, shapes = TT_SHAPES, in_dims = [32, 48, 2], ranks = TT_RANKS)
            colbert.bert.encoder.layer[i].output.dense = factorized_layer

print(sum(p.numel() for p in colbert.parameters()))

layer.shape torch.Size([3072, 768])
torch.Size([1, 32, 32])
torch.Size([32, 48, 450])
torch.Size([450, 48, 32])
torch.Size([32, 32, 1])
layer.shape torch.Size([768, 3072])
layer.shape torch.Size([3072, 768])
torch.Size([1, 32, 32])
torch.Size([32, 48, 450])
torch.Size([450, 48, 32])
torch.Size([32, 32, 1])
layer.shape torch.Size([768, 3072])
layer.shape torch.Size([3072, 768])
torch.Size([1, 32, 32])
torch.Size([32, 48, 450])
torch.Size([450, 48, 32])
torch.Size([32, 32, 1])
layer.shape torch.Size([768, 3072])
layer.shape torch.Size([3072, 768])
torch.Size([1, 32, 32])
torch.Size([32, 48, 450])
torch.Size([450, 48, 32])
torch.Size([32, 32, 1])
layer.shape torch.Size([768, 3072])
layer.shape torch.Size([3072, 768])
torch.Size([1, 32, 32])
torch.Size([32, 48, 450])
torch.Size([450, 48, 32])
torch.Size([32, 32, 1])
layer.shape torch.Size([768, 3072])
layer.shape torch.Size([3072, 768])
torch.Size([1, 32, 32])
torch.Size([32, 48, 450])
torch.Size([450, 48, 32])
torch.Size([32, 32, 1])
laye

In [5]:
v = torch.rand(256, 142, 768)

In [6]:
out = factorized_layer(v)
out.shape

input shape torch.Size([256, 142, 768])
shapes, in_dims, ranks (32, 48, 48, 32) [32, 24] [1, 380, 390, 380, 1]
256 142
result.shape, core.shape torch.Size([36352, 32, 24]) torch.Size([32, 32])
result.shape torch.Size([36352, 24, 32])
result.shape, core.shape torch.Size([36352, 24, 32]) torch.Size([32, 24, 2, 390])
result.shape torch.Size([36352, 2, 390])
result.shape, core.shape torch.Size([256, 284, 390]) torch.Size([390, 48, 32])
result.shape torch.Size([256, 284, 48, 32])
result.shape, core.shape torch.Size([256, 13632, 32]) torch.Size([32, 32, 1])
result.shape torch.Size([256, 13632, 32, 1])


torch.Size([256, 142, 3072])

In [7]:
U, S, Vt = np.linalg.svd(fc_w, full_matrices=False)


# truncate SVD and fuse Sigma matrix
w1 = np.dot(np.diag(np.sqrt(S[0:rank])),Vt[0:rank, :])
w2 = np.dot(U[:, 0:rank], np.diag(np.sqrt(S[0:rank])))

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [2]:
print(sum(p.numel() for p in colbert.parameters()))

109580544


In [3]:
#from colbert.utils.easytt import TTLayer

In [4]:
32, 48, 48, 32

(32, 48, 48, 32)

In [5]:
import tntorch as tn
import torch
import time

import numpy as np
X, Y, Z, W = np.meshgrid(range(128), range(128), range(128), range(128))
full = torch.randn((128, 128, 128, 128))  # Some analytical 3D function
print(full.shape)

torch.Size([128, 128, 128, 128])


In [6]:
import tensorly as tl
from tensorly.decomposition import tensor_train
tl.set_backend('pytorch')

t = tensor_train(full, rank = [1, 130, 130, 130, 1])



/opt/.pyenv/versions/3.8.10/lib/python3.8/site-packages/tensorly/backend/core.py:1106: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


In [7]:
shapes = [[32, 48], [48, 32]]

In [8]:
shapes = np.asarray(shapes)
print ("shapes", shapes)
print (list(shapes.flatten()))
matrix = colbert.bert.encoder.layer[1].intermediate.dense.weight.reshape(list(shapes.flatten()))
print (matrix.shape)  
d = len(shapes[0])
transpose_idx = list(np.arange(2 * d).reshape(2, d).T.flatten())
matrix = matrix.permute(*transpose_idx)
newshape = np.prod(shapes, 0)
matrix = matrix.reshape(list(newshape))
ranks = [530, 540, 530]
tt = tn.Tensor(matrix, ranks_tt=ranks)

shapes [[32 48]
 [48 32]]
[32, 48, 48, 32]
torch.Size([32, 48, 48, 32])


AssertionError: 

In [ ]:
tt

In [7]:
list(np.arange(2 * 1).reshape(2, 1).T.flatten())

[0, 1]

In [8]:
print ("matrix.shapes", matrix.shape)
shapes = np.asarray(shapes)
print ("shapes", shapes)
print (list(shapes.flatten()))
matrix = matrix.reshape(list(shapes.flatten()))
print (matrix.shape)  
d = len(shapes[0])
transpose_idx = list(np.arange(2 * d).reshape(2, d).T.flatten())
matrix = matrix.permute(*transpose_idx)
newshape = np.prod(shapes, 0)
matrix = matrix.reshape(list(newshape))
tt = tn.Tensor(matrix, ranks_tt=ranks)


TypeError: __init__() got an unexpected keyword argument 'in_modes'

In [ ]:
import numpy as np
import torch
from torch import nn
import tntorch as tn


def matrix_to_tt_cores(matrix, shapes, ranks):
  print ("matrix.shapes", matrix.shape)
  shapes = np.asarray(shapes)
  print ("shapes", shapes)
  print (list(shapes.flatten()))
  matrix = matrix.reshape(list(shapes.flatten()))
  print (matrix.shape)  
  d = len(shapes[0])
  transpose_idx = list(np.arange(2 * d).reshape(2, d).T.flatten())
  matrix = matrix.permute(*transpose_idx)
  newshape = np.prod(shapes, 0)
  matrix = matrix.reshape(list(newshape))
  #tt = tn.Tensor(matrix, ranks_tt=ranks)
  tt = tensor_train(full, rank = ranks)


  newcores = []
  for core, s1, s2, r1, r2 in zip(tt.cores,
                                  shapes[0], shapes[1],
                                  tt.ranks_tt, tt.ranks_tt[1:]):
    newcores.append(core.reshape((r1, s1, s2, r2)))
  return newcores


def ttmatmul(cores, t, shapes, ranks):
  ranks = [1] + ranks + [1]
  tshape = t.shape

  t = t.transpose(1, 0)
  t = t.reshape((-1, shapes[1][-1], 1))
  ndims = len(cores)
  for i in reversed(range(ndims)):
    t = torch.einsum('aijb,rjb->ira', (cores[i], t))
    if i:
      t = t.reshape((-1, shapes[1][i - 1], ranks[i]))
  t = t.reshape((int(np.prod(shapes[0])), tshape[1]))
  return t


def transpose(cores):
  result = []
  for c in cores:
    result.append(c.permute((0, 2, 1, 3)))
  return result


def matmultt(t, cores, shapes, ranks):
  t = t.transpose(1, 0)
  cores = transpose(cores)
  shapes = [shapes[1], shapes[0]]
  return ttmatmul(cores, t, shapes, ranks).transpose(1, 0)


class TTLayer(nn.Module):
  def __init__(self, layer, shapes, ranks):
    super(TTLayer, self).__init__()
    self.shapes = shapes
    self.ranks = ranks
    with torch.no_grad():
      weight = layer.weight.transpose(1, 0).data.cpu()
      print ("weight shape", weight.shape)
      self.cores = nn.ParameterList(
          map(nn.Parameter, matrix_to_tt_cores(weight, shapes, ranks)))
    self.bias = layer.bias

  def forward(self, inputs):
    out = matmultt(inputs, self.cores, self.shapes, self.ranks)
    out = out + self.bias
    return out


In [ ]:
from src.classes.gpt2_tt import GPT2_TT_Model
from src.classes.gpt_med_config import GPT2MedConfig

rank = 70

for i in [0, 2, 4, 6, 8, 10]:
    # fc part
    fc_w = colbert.bert.encoder.layer[i].intermediate.dense
    print (fc_w.weight.shape)
    fc_b = colbert.bert.encoder.layer[i].intermediate.dense.bias.data.cpu()
    factorized_layer = TTLayer(fc_w, shapes = [[32, 48], [48, 32]], ranks = [1, 530, 540, 530, 1])
    print (len(factorized_layer.ttm.tt.cores))
    for elem in factorized_layer.ttm.tt.cores:
        print (elem.shape)
    colbert.bert.encoder.layer[i].intermediate.dense = factorized_layer

In [17]:
from src.classes.gpt2_tt import GPT2_TT_Model
from src.layers2.linear import TTMLinear
from src.classes.gpt_med_config import GPT2MedConfig

from src.ttm_linear.ttm_linear.ttm_linear import FactorizationTTMLinear
rank = 50

for i in [0, 2, 4, 6, 8, 10]:
    # fc part
    fc_w = colbert.bert.encoder.layer[i].intermediate.dense.weight.data
    fc_b = colbert.bert.encoder.layer[i].intermediate.dense.bias.data
    (in_, out_) = fc_w.shape
    factorized_layer = FactorizationTTMLinear(in_, out_, rank=rank, max_core_dim_product = rank)
    factorized_layer.fill_with_pretrained_matrix(fc_w, reshape_sizes = (8, 12, 8, 16, 12, 16))
    print (len(factorized_layer.ttm.tt.cores))
    for elem in factorized_layer.ttm.tt.cores:
        print (elem.shape)
    colbert.bert.encoder.layer[i].intermediate.dense = factorized_layer
print(sum(p.numel() for p in colbert.parameters()))

-------------------------------------
TTM-Linear required dimensions: dim_in=3072, dim_out=768, rank=50, max_dim=50
    after best_approx: dim_in=3072, dim_out=768
    dim_in factorization:  (2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3)
    dim_out factorization: (2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3)
    dims before shrink:  [(2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 3), (2, 1), (3, 1)]
    final TTM dims:  [(8, 8), (8, 8), (8, 12), (6, 1)]
ranks [1, 50, 50, 50, 1]
dims 1 8 8 50 3200
dims 50 8 8 50 160000
dims 50 8 12 50 240000
dims 50 6 1 1 300
self.dims [(8, 8), (8, 8), (8, 12), (6, 1)]
ranks in TTM [50, 50, 50]
    Original linear params: 2359296, ttm params: 320000 (x0.136)
-------------------------------------
torch.Size([8, 8, 8, 6, 8, 8, 12, 1]) torch.Size([8, 8, 8, 6, 8, 8, 12, 1])
tensor(1.9725)
4
torch.Size([1, 8, 8, 50])
torch.Size([50, 8, 8, 50])
torch.Size([50, 8, 12, 6])
torch.Size([6, 6, 1, 1])
self.dims [(8, 8), (8, 8), (8, 12), (6, 1)]
ranks in TTM [50, 50, 5

In [9]:
print(sum(p.numel() for p in colbert.parameters()))

99752064
